<a href="https://colab.research.google.com/github/Christian-Albertini/project-4/blob/main/Project_4_high_recall.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import dependencies
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import roc_auc_score
from imblearn.over_sampling import SMOTE
import tensorflow as tf
import numpy as np



# Use raw URLs from GitHub
combined_url = 'https://raw.githubusercontent.com/Christian-Albertini/project-4/main/Resources/Combined_Player_Stats_Cleaned_3.csv'

# Read the CSV files
Combined_Player_Stats = pd.read_csv(combined_url)

# Display the first few rows of each DataFrame
Combined_Player_Stats.head()


,Season,Team,League,Games_Played,All-Star,Plate_Appearances,At_Bats,Runs,Hits,Single,...,Stolen_Bases,Caught_Stealing,Walks,Strikeouts,Batting_Average,On_Base_Percentage,Slugging_Percentage,On_Base_Plus_Slugging_(OPS),On_Base_Plus_Slugging_Plus_(OPS+),Total_Bases
0,2023,BOS,AL,92,0,353,320,45,79,32,...,4,0,22,110,0.247,0.303,0.531,0.834,119,170
1,2023,BAL,AL,141,0,455,412,59,99,63,...,11,4,32,68,0.240,0.300,0.396,0.696,94,163
2,2023,BAL,AL,154,1,687,588,84,163,111,...,1,2,92,101,0.277,0.374,0.435,0.809,128,256
3,2023,TEX,AL,148,1,632,555,108,136,68,...,9,1,65,175,0.245,0.328,0.508,0.836,127,282
4,2023,DET,AL,112,0,357,312,40,68,43,...,14,3,42,89,0.218,0.310,0.372,0.682,88,116


In [2]:
# Convert categorical data to numeric
combined_df = pd.get_dummies(Combined_Player_Stats, columns=['League'])
combined_df=combined_df.drop(columns=['Team'])

# Review Dataframes
combined_df.head()

,Season,Games_Played,All-Star,Plate_Appearances,At_Bats,Runs,Hits,Single,Double,Triple,...,Walks,Strikeouts,Batting_Average,On_Base_Percentage,Slugging_Percentage,On_Base_Plus_Slugging_(OPS),On_Base_Plus_Slugging_Plus_(OPS+),Total_Bases,League_AL,League_NL
0,2023,92,0,353,320,45,79,32,24,2,...,22,110,0.247,0.303,0.531,0.834,119,170,True,False
1,2023,141,0,455,412,59,99,63,21,2,...,32,68,0.240,0.300,0.396,0.696,94,163,True,False
2,2023,154,1,687,588,84,163,111,31,1,...,92,101,0.277,0.374,0.435,0.809,128,256,True,False
3,2023,148,1,632,555,108,136,68,29,0,...,65,175,0.245,0.328,0.508,0.836,127,282,True,False
4,2023,112,0,357,312,40,68,43,13,1,...,42,89,0.218,0.310,0.372,0.682,88,116,True,False


In [3]:
# Identify feature and target arrays
y = combined_df['All-Star']
X = combined_df.drop(columns=['All-Star'])

# Split the data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit and scale the data
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Apply SMOTE to balance the dataset
smote = SMOTE()
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

In [6]:
# Compute class weights
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train_resampled),
    y=y_train_resampled
)
class_weight_dict = dict(enumerate(class_weights))

# Define model
number_input_features = len(X_train_resampled[0])
hidden_nodes_layer1 = 50
hidden_nodes_layer2 = 25
hidden_nodes_layer3 = 10

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation='relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='sigmoid'))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation='sigmoid'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [7]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [8]:
# Train the model
fit_model = nn.fit(
    X_train_resampled, y_train_resampled,
    epochs=50,
    validation_split=0.2,
    class_weight=class_weight_dict,
    verbose=1
)

Epoch 1/50
71/71 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.6170 - loss: 0.6412 - val_accuracy: 0.0000e+00 - val_loss: 0.8204
Epoch 2/50
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7186 - loss: 0.5260 - val_accuracy: 0.7588 - val_loss: 0.7089
Epoch 3/50
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7860 - loss: 0.4847 - val_accuracy: 0.8116 - val_loss: 0.6277
Epoch 4/50
71/71 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7980 - loss: 0.4635 - val_accuracy: 0.7940 - val_loss: 0.6165
Epoch 5/50
71/71 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8039 - loss: 0.4454 - val_accuracy: 0.7782 - val_loss: 0.6169
Epoch 6/50
71/71 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7878 - loss: 0.4512 - val_accuracy: 0.7887 - val_loss: 0.5707
Epoch 7/50
71/71 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8163 - loss: 0.4090 - val_accuracy: 0.8011 - val_loss: 0.5352
Epoch 8/50
71/71 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8137 - loss: 0.4077 - val_accuracy: 0.7817 - val_

In [9]:
# Predict probabilities
y_pred_proba = nn.predict(X_test_scaled)

# Convert probabilities to binary class labels
y_pred = (y_pred_proba > 0.5).astype('int32').flatten()

# Generate and print classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Print ROC-AUC score
roc_auc = roc_auc_score(y_test, y_pred_proba)
print(f"ROC-AUC Score: {roc_auc}")

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.82      0.87       368
           1       0.41      0.63      0.50        71

    accuracy                           0.79       439
   macro avg       0.67      0.73      0.68       439
weighted avg       0.84      0.79      0.81       439

ROC-AUC Score: 0.8217620943049602
